##Imports


In [1]:
!pip install opencv-python numpy matplotlib open3d


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.

### Preprocess Images
Load and preprocess the images to ensure they are in the correct format for further processing. This includes resizing and normalizing them.

In [2]:
import cv2

# Load images
front_view = cv2.imread('/content/front.jpg')
side_view = cv2.imread('/content/side.jpg')
top_view = cv2.imread('/content/top.jpg')

# Resize images if necessary
height, width = 480, 640  # Example dimensions
front_view = cv2.resize(front_view, (width, height))
side_view = cv2.resize(side_view, (width, height))
top_view = cv2.resize(top_view, (width, height))


 ### Estimate Depth Maps
 Use a depth estimation model to create depth maps from each of the images.

In [3]:
def estimate_depth(image):
    import numpy as np
    import cv2

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Use a Sobel filter to approximate gradients (edges can help infer depth)
    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=5)
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=5)

    # Combine gradients to simulate depth intensity (simplistic approach)
    depth_map = np.sqrt(sobel_x**2 + sobel_y**2)


    # Normalize depth map to scale 0-255
    depth_map = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    return depth_map

depth_front = estimate_depth(front_view)
depth_side = estimate_depth(side_view)
depth_top = estimate_depth(top_view)


###  Create Point Clouds
Convert the depth maps into point clouds. A point cloud is a collection of points in space representing the external surface of an object.


In [7]:
import numpy as np
def create_point_cloud(image, depth_map):
    # Get image dimensions
    h, w = image.shape[:2]

    # Generate point cloud
    points = []
    for y in range(h):
        for x in range(w):
            z = depth_map[y, x]  # Depth value at pixel (x,y)
            if z > 0:  # Only consider points with valid depth values
                points.append([x, y, z])

    return np.array(points)

point_cloud_front = create_point_cloud(front_view, depth_front)
point_cloud_side = create_point_cloud(side_view, depth_side)
point_cloud_top = create_point_cloud(top_view,depth_top)

### Merge Point Clouds
 Combine the point clouds generated from each view into a single unified point cloud.

In [8]:
combined_point_cloud = np.vstack((point_cloud_front, point_cloud_side, point_cloud_top))

 ### Create 3D Mesh
Using Open3D or similar libraries, convert the combined point cloud into a mesh.

In [9]:
import open3d as o3d

# Create Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_point_cloud)

### Estimate normals and create mesh

In [10]:
pcd.estimate_normals()
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd)

### Visualize the mesh

In [11]:

o3d.visualization.draw_geometries([mesh])


[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


### Save the Model
Finally, save your generated 3D model in a suitable format (like .obj or .ply) for further use or analysis.


In [12]:
o3d.io.write_triangle_mesh("output_model.ply", mesh)



True

# Conclusion

1.This workflow provides a basic framework for performing 3D reconstruction from three different views using Python.    
2.You may need to adapt certain parts based on specific requirements or enhancements like texture mapping or more sophisticated depth estimation techniques.




